In [19]:
import datetime
from ib_insync import *
import pandas as pd
import csv
import os
import time as tyme
import numpy as np
from arctic import Arctic
from arctic import TICK_STORE
from multiprocessing import Process
import time as tyme
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.graph_objs
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
from IPython.display import clear_output, display


In [4]:
base_dir = '/Users/thomasmbird/Documents/Quant Stuff/LiveMktData/Commodities/'
files = os.listdir(base_dir)
print(files)

['HON4', 'RBK4', 'PLX3', 'HOM4', 'CLU4', 'RBH4', 'NGQ4', 'HOG4', 'GCF4', 'CLG4', 'RBZ3', 'HGZ3', 'HOU4', '.DS_Store', 'CLM4', 'PLJ4', 'SIX3', 'NGJ4', 'CLN4', 'CLJ4', 'NGN4', 'HOQ4', 'NGG4', 'NGM4', 'GCZ3', 'RBF4', 'HGF4', 'NGU4', 'CLQ4', 'HOJ4', 'HOK4', 'SILX3', 'RBN4', 'SIF4', 'RBG4', 'CLZ3', 'HGG4', 'HOH4', 'RBM4', 'NGF4', 'RBU4', 'CLH4', 'PLF4', 'HOZ3', 'CLK4', 'GCX3', 'HGX3', 'NGK4', 'SIZ3', 'CLF4', 'NGH4', 'RBQ4', 'NGZ3', 'SIH4', 'PLZ3', 'HOF4', 'RBJ4']


In [42]:
contr = 'NGZ3'
base_dir = '/Users/thomasmbird/Documents/Quant Stuff/LiveMktData/Commodities/'
file = base_dir+'/'+contr+'/top_of_book_11_15_23.csv'
tob_data = pd.read_csv(file,index_col='time',parse_dates=['time'])
tob_data = tob_data.dropna()
tob_data = tob_data.drop(tob_data[tob_data['bid'] == -1].index)
tob_data = tob_data.drop(tob_data[tob_data['ask'] == -1].index)

#print(tob_data.tail(5).to_string())

tob_data['MMA30']= (tob_data['bid']/2+tob_data['ask']/2).rolling(window=30,min_periods=1).mean()

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                                vertical_spacing=0.03, subplot_titles=(contr+'Bid/Ask Spread', contr+'Top of Book Bid/Ask Size'),
                                row_width=[0.2, 0.7])
            
fig.add_trace(go.Scatter(x=tob_data.index,y=tob_data['bid'],mode='markers',name='Bid',
                         marker=dict(
    color='Green',
    size=2,
    )),
              row=1, col=1)
fig.add_trace(go.Scatter(x=tob_data.index,y=tob_data['ask'],mode='markers',name='Ask',
                         marker=dict(
    color='Red',
    size=2,
    )),
              row=1, col=1)
fig.add_trace(go.Scatter(x=tob_data.index,y=tob_data['MMA30'],mode='markers+lines',name='30 min MA',
                         marker=dict(
    color='Black',
    size=1,
    )),
              row=1, col=1)



#fig.add_trace(go.Candlestick(x=es_df.index, open=es_df['open'],
#                   high=es_df['high'], low=es_df['low'], close=es_df['close'],
#                             name="OHLC"),
#              row=1, col=1)
#fig.add_trace(go.Scatter(x=es_df.index,y=es_df['MMA30'],mode='markers',
#                         marker=dict(
#    color='LightSkyBlue',
#    size=8,
#    )),
#              row=1, col=1)
#fig.update_traces(marker_color='blue')
#fig.update(layout_xaxis_rangeslider_visible=False)

################################################################################################################################################
######        Stacked Bar graphs for volume
################################################################################################################################################

fig.add_trace(go.Bar(x=tob_data.index, y=tob_data['askSize'], marker = dict(color='#d62728'), name='Ask Size'),
              row=2, col=1)
fig.add_trace(go.Bar(x=tob_data.index, y=tob_data['bidSize'], marker = dict(color='#2ca02c'), name='Bid Size'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=tob_data.index,y=(tob_data['askSize']+tob_data['bidSize']),mode='markers+lines',name='Total Size',
                          marker=dict(
     color='Black',
     size=1,
     )),
               row=2, col=1)

fig.update_layout(barmode = 'stack')

fig.update_traces(
      marker_line_width = 0,
      selector=dict(type="bar"))

fig.update_layout(bargap=0,
                  bargroupgap = 0,
                 )




#fig.update_traces(marker_color='rgba(0,0,250, 0.5)',
#                  marker_line_width=0,
#                  selector=dict(type="bar"))
fig.update_layout({'autosize': False, 'height':800,'width':1200})
clear_output(wait=True)
        #fig.show()
        
display(fig)
print(tob_data['lastSize'].sum())

9654.0


In [47]:
base_dir = '/Users/thomasmbird/Documents/Quant Stuff/LiveMktData/Rates/'
files = os.listdir(base_dir)
#print files
#bid_df = pd.DataFrame(columns=['maxBid','medBid','meanBid','maxAsk','medAsk','meanAsk','tot'])
bid_df = pd.DataFrame(columns=['volume'])
for folder in files:
    if 'HG' in folder:
        diff_files = os.listdir(base_dir+folder)
        volsum = 0
        for file in diff_files:
            data = pd.read_csv(base_dir+folder+'/'+file,index_col='timestamp',parse_dates=['timestamp'])
            volsum += data['volume'].sum()
            
        bid_df.loc[folder] = [volsum]
        #bid_df.loc[folder] = [data['bidSize'].max(),data['bidSize'].median(),data['bidSize'].mean(),
        #                      data['askSize'].max(),data['askSize'].median(),data['askSize'].mean(),
        #                      data['askSize'].mean()+data['bidSize'].mean()]
        
        
print(bid_df.sort_values(by=['volume'],ascending=False).to_string())

        volume
HGZ3  407137.0
HGF4     593.0
HGG4     360.0
HGX3     334.0
